In [1]:
from sentence_transformers import SentenceTransformer
#ST = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
ST = SentenceTransformer('bongsoo/moco-sentencedistilbertV2.1')


/root/anaconda3/envs/naver/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/root/anaconda3/envs/naver/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
"""

from sentence_transformers import SentenceTransformer
sentences = ["서울은 한국이 수도이다", "The capital of Korea is Seoul"]

model = SentenceTransformer('bongsoo/moco-sentencedistilbertV2.1')
embeddings = model.encode(sentences)
print(embeddings)

# sklearn 을 이용하여 cosine_scores를 구함
# => 입력값 embeddings 은 (1,768) 처럼 2D 여야 함.
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
cosine_scores = 1 - (paired_cosine_distances(embeddings[0].reshape(1,-1), embeddings[1].reshape(1,-1)))

print(f'*cosine_score:{cosine_scores[0]}')

"""


In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def search(query:str,k:int, database:list):

    embedded_query = ST.encode(query)
    
    embedded_database = [ST.encode(doc) for doc in database]
    
    similarities = [cosine_similarity([embedded_query], [doc_emb])[0][0] for doc_emb in embedded_database]
    
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    top_k_elements = [database[idx] for idx in top_k_indices]
    
    return top_k_elements


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id="MLP-KTLim/llama-3-Korean-Bllossom-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.25s/it]


In [8]:
SYS_PROMPT = """당신은 Question에 답변을 제공하는 도우미입니다.
Context 내의 예시들을 참고하여서 Question에 답하시오"""

In [9]:
def format_prompt(prompt,retrieved_documents,k):
  PROMPT = f"Question:{prompt}\nContext:"
  for idx in range(k) :
    PROMPT+= f"{retrieved_documents[idx]}\n"
  return PROMPT

def generate(formatted_prompt):
    formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM
    messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)


In [10]:
def rag_chatbot(prompt,k,database):

  retrieved_documents = search(prompt, k,database=database)
  formatted_prompt = format_prompt(prompt,retrieved_documents,k)
  result = generate(formatted_prompt)
  
  return result

In [11]:

database=["딸기","사과"] 

rag_chatbot("음식을 추천해주시오", k = 1,database=database)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'사과에게 추천할 음식은 다양할 수 있지만, 일반적으로 사과는 신선하고 건강한 음식을 좋아합니다. 다음은 사과를 위한 몇 가지 음식 추천입니다:\n\n1. **과일 잼**: 과일 잼은 사과에게 매우 유리한 음식입니다. 과일 잼은 과일의 씨가 들어가며, 사과에게 중요한 비타민과 철분을 공급합니다.\n\n2. **과일잼**: 과일잼은 과일 잼의 형태가 다를 뿐이지, 과일의 씨가 들어가며 비슷한 영양 효과를 제공합니다.\n\n3. **과일즙**: 과일즙은 과일의 잎가 들어가며, 사과에게 중요한 비타민과 철분을 공급합니다.\n\n4. **과일즙**: 과일즙은 과일의 잎가 들어가며, 사과에게 중요한 비타민과 철분을 공급합니다.\n\n5. **과일즙**: 과일즙은 과일의 잎가 들어가며, 사과에게 중요한 비타민과 철분을 공급합니다.\n\n이 음식들은 사과에게 건강한 영양을 제공하며, 사과의 생명을 유지하는 데미치 않은 것입니다.'